## Preprocessing

In [4]:
import sys
import os
sys.path.append('../')
from utils import *
import pandas as pd
pd.set_option('display.max_columns', None)
pd.option_context("display.max_rows", None)
import numpy as np
from sklearn import preprocessing as ps
import warnings
warnings.filterwarnings("ignore")
from sklearn.impute import KNNImputer
import joblib
from scipy import stats
from scipy.special import inv_boxcox

In [5]:
df = pd.read_excel('../../01_Data/02_Processed/processed_test.xlsx')
ID = df.SUBMISSION_ID

In [6]:
df[['TOP_FMLS','pd_bi','FMLS_PD_PERCEN','FMLS_BI_PERCEN']] = df[['TOP_FMLS','pd_bi',
                                                                 'FMLS_PD_PERCEN','FMLS_BI_PERCEN']].fillna(0)

In [7]:
f = df[['MAIN_PRICING_CATEG','MAIN_PRICING_SUBCATEG','SUBSECTOR']]
f2 = df[['MARKET','MAIN_PRICING_SUBCATEG']]
feature_hash(df, f, max_size=60)
feature_hash(df, f2, max_size=30)
df.drop(['MARKET','MAIN_PRICING_CATEG','MAIN_PRICING_SUBCATEG','SUBSECTOR'], axis=1, inplace=True)

encoder = ps.LabelEncoder()
df['UWCENTER'] = encoder.fit_transform(df['UWCENTER'])
df['SEGMENT_LOB'] = encoder.fit_transform(df['SEGMENT_LOB'])
df['CT_STATUS'] = encoder.fit_transform(df['CT_STATUS'])
df['COVERAGE'] = encoder.fit_transform(df['COVERAGE'])
df['BUSINESSUNIT'] = encoder.fit_transform(df['BUSINESSUNIT'])
df['UF_STATUS'] = encoder.fit_transform(df['UF_STATUS'])

In [8]:
df.drop(['MAINOCCUPANCY','OIL','NO_OF_STRY','pd_bi','FXRATEUSD','FMLS_BI','IV_CONTENT',
         'DISCOUNTS','ORIGCCY','WORDING','QUALITY_RISK_MGT','PARTTYPE','FMLS_BI_PERCEN',
         'MB_QUALITY','MAIN_COUNTRY','GUARANTEE','TAG','inception_year','BI_TYPE','inception_duration','inception_month',
         'COVERAGE','YEAR','nb_category','BI_MITIGATION','ASSET_QUALITY','SEGMENT_LOB','CT_STATUS','YR_BLT',
         'NO_OF_BLDG','FLD_EXP','MHZ_EXP','CYC_EXP','CYCPRO_EXP','MHZPRO_EXP','VOL_EXP',
        'UF_STATUS','FMLS_PD_PERCEN','TIV','ACCURACY_VALIDATE','FMLS_PD','MPL_PD','NBLOCS','IV_BI_VAL',
         'diff','MAIN_PRICING_SUBCATEG_hash','IV_PD','SUBSECTOR_hash','BI_PERIOD','CT_PERIOD','nb_state',
         'TOP_FMLS','MPL_BI_PERCEN','UWY_first','UWR_cumultive','pricing_duration'], axis=1,inplace=True)

In [9]:
df.drop(['SUBMISSION_ID','FACUL_NUM','DIVISION_NUM','UWYEAR'], axis=1, inplace=True)

In [10]:
df.shape

(3259, 24)

## Predict on test set

In [12]:
knn_imp = KNNImputer(n_neighbors=4, weights='distance')
df_impute = pd.DataFrame(knn_imp.fit_transform(df), columns=df.columns)

In [13]:
cb = joblib.load('../../03_Model/cb_reg_030821.pkl')
pred_cb = cb.predict(df_impute)

In [14]:
xgb = joblib.load('../../03_Model/xgb_reg_030821.pkl')
org_cols_order = xgb.get_booster().feature_names
df_impute_2 = df_impute[org_cols_order]
pred_xgb = xgb.predict(df_impute_2)

In [15]:
stack = joblib.load('../../03_Model/stack_reg_030821_best.pkl')
pred_stack = stack.predict(df_impute)

In [16]:
pred_cb_df = pd.DataFrame(pred_cb, columns=['predicted_premium'])
result_cb = pd.concat([ID, pred_cb_df], names=['SUBMISSION_ID','predicted_premium'], axis=1)

pred_xgb_df = pd.DataFrame(pred_xgb, columns=['predicted_premium'])
result_xgb = pd.concat([ID, pred_xgb_df], names=['SUBMISSION_ID','predicted_premium'], axis=1)

pred_stack_df = pd.DataFrame(pred_stack, columns=['predicted_premium'])
result_stack = pd.concat([ID, pred_stack_df], names=['SUBMISSION_ID','predicted_premium'], axis=1)

In [17]:
result_cb['predicted_premium'] = inv_boxcox(result_cb['predicted_premium'], 0.17700863235182274)
result_xgb['predicted_premium'] = inv_boxcox(result_xgb['predicted_premium'], 0.17700863235182274)
result_stack['predicted_premium'] = inv_boxcox(result_stack['predicted_premium'], 0.17700863235182274)

## Save result

In [18]:
result_cb.to_csv("../../01_Data/03_Results/result_premium_catboost_final.csv",index=False)
result_xgb.to_csv("../../01_Data/03_Results/result_premium_xgb_final.csv",index=False)
result_stack.to_csv("../../01_Data/03_Results/result_premium_stack_final_best.csv",index=False)